In [1]:
rm(list=ls())

#library(openair)
library(parallel)

setwd("/home/jrosas/SMAP_CLOUD/SMAP_DESK_COURSE/EsquemaSMAP_nuvem/")

source("functions_SMAP_novoVies_parallel.R")

In [10]:
load("Parametros.RData")

#datename <- "xtodayx"
date_file <- "20200624"
datename <- date_file

subname <- vertudo$subbacia
bigname <- vertudo$bacia

namesObs <- c("QQueixo","ESPORA", "QM", "Monjolinho", "StaClara", "EDACUNHA", "ESouza", "JordSeg", "FozChapeco", "Machadinho", "CAPESCURO",
              "FOA", "Camargos", "PBUENOS", "CORUMBAIV", "FUNIL MG", "PARAGUACU", "Chavantes", "SDO","SJoao", "SDOFACAO", "RB-SMAP", 
              "Ibitinga","SALTOVERDI","BG", "Ita", "CanoasI", "Flor+Estra", "CN", "Maua", "NOVAPONTE", "Rosana", "PASSAGEM", "NAvanhanda", 
              "Jurumirim", "SCaxias", "EMBORCACAO", "PCOLOMBIA", "MARIMBONDO", "FURNAS", "CORUMBA1", "AVERMELHA", "Balsa", "ITUMBIARA",
              "UVitoria", "Capivara", "BBonita","JUPIA", "SFR2", "RVerde", "Ivinhema", "PTaquara", "TM-SMAP","ILHAEQUIV", "SSIMAO2", "Itaipu", "SMesa","PPRI","SRM2")


folder <- paste("Modelos_Chuva_Vazao_", date_file, sep="")

In [5]:
folder <- paste("Modelos_Chuva_Vazao_", date_file, sep="")
saidas_bkup <- list()
lag<-1
tstart <- Sys.time()
mainONS <- function(datename, folder, subname, bigname, namesObs, vertudo,lag,i){

  setwd("/home/jrosas/SMAP_CLOUD/SMAP_DESK_COURSE/EsquemaSMAP_nuvem/")
  print(subname)
  dir <- paste(folder, "/SMAP/", bigname, "/ARQ_ENTRADA/", sep="")
  modname <- "GEFS"
  if(length(Sys.glob(paste(dir, "*", sep=""))) > 0){
    
    valoresInicializacao <- readInic(dir, subname)
    datei <- valoresInicializacao$date
    diasWarm <- as.integer(valoresInicializacao$infos[1])
#   diasPrev <- as.integer(valoresInicializacao$infos[2])
    diasPrev <- 12
    ebin <- valoresInicializacao$infos[3]
    supin <- valoresInicializacao$infos[4]
    tuin <- valoresInicializacao$infos[5]
    
    mod <- geraSerieChuvaModel(dir, subname, modname)  
    obs <- geraSerieObservada(dir, subname)   # Observed rainfall serie
     
    # Set the parameters to the especific sub-basin
    param <- vertudo[vertudo$subbacia == subname,-1]
    y <- param[, substr(names(param),1,3) == "kt_"]
    y <- y[!is.na(y)]
    y <- length(y)-3
    
    # Applies the time coefficients and pcoef to the rain series (for optimization)
    df <- geraChuvaSMAP(datei, diasWarm, diasPrev, mod, obs, vertudo, param$pcof, subname, y, lag)
    df_previsao <- df[[2]]
    df <- df[[1]]
    
    # Opens the streamflow observed data
    vaz <- leVazaoObs(dir, subname)
    vaz <- merge(vaz, df, all.y=T)[,c(1,2)]  # set the same time period to optimization
    vaz <- vaz[!is.na(vaz$vazaoObs),]
    
    # Opens the evapotranspiration serie
    evapo <- read.table(paste(dir,subname,"_EVAPOTRANSPIRACAO.txt", sep=""))
    evapo_prev <- evapo[as.numeric(format(df_previsao$date, "%m")),2]
    evapo <- evapo[as.numeric(format(df$date, "%m")),2]
    
    bat_pars <- readBat(dir)
    Lower = c(rep(param$inf_chuva,31), param$inf_ebin*ebin, 0)
    Upper = c(rep(param$sup_chuva,31), param$sup_ebin*ebin, 1000000)
    cal <- bat_optim_leo(D=33, NP=as.numeric(bat_pars[4,2]), N_Gen=5000, A=as.numeric(bat_pars[1,2]), gamma=as.numeric(bat_pars[5,2]), Lower, Upper, tuin,
                         param, df, evapo, vaz$vazaoObs,
                         alvo=as.numeric(bat_pars[2,2]), semente=as.numeric(bat_pars[11,2]), constA=as.numeric(bat_pars[6,2]), 
                         constB=as.numeric(bat_pars[7,2]), ipeso=bat_pars[9,2], tipo=bat_pars[8,2])
    
    
    setwd(dir)
    
    chuvafinal <- rbind(df, df_previsao)
    evapofinal <- c(evapo, evapo_prev)
    pesosfinal <- c(cal[1:31], rep(1,length(df_previsao$date)))
    
    
    qprev <- data.frame(date=chuvafinal$date, vaz=smap(tuin, cal[32], cal[33],
                                                       param$Area,
                                                       param$kkt, param$k1t, param$k2t, param$k2t2, param$k3t,
                                                       param$str, param$crec, param$ai, param$capc,
                                                       param$H, param$H1,
                                                       param$pcof, param$ecof, param$ecof2,
                                                       evapofinal, chuvafinal,
                                                       pesosfinal,
                                                       0))
    
    #qprev <- qprev[,1:2]
    names(qprev)[2] <- subname
    
    return(qprev[,1:2])
    
  }
  
  
}


In [17]:

x<-40
mainONS(datename, folder, subname[x], bigname[x], namesObs[x], vertudo,lag,x)



[1] "MARIMBONDO"
[1] "240620"
[1] 12
          date    mediaT
1   2020-05-21 0.0000000
2   2020-05-22 0.0000000
3   2020-05-23 3.2735500
4   2020-05-24 3.6201750
5   2020-05-25 1.0610250
6   2020-05-26 0.6556500
7   2020-05-27 0.0000000
8   2020-05-28 0.0000000
9   2020-05-29 0.0000000
10  2020-05-30 0.0000000
11  2020-05-31 0.0000000
12  2020-06-01 0.0000000
13  2020-06-02 0.0000000
14  2020-06-03 0.1727250
15  2020-06-04 0.1727250
16  2020-06-05 0.1480500
17  2020-06-06 1.2666500
18  2020-06-07 2.0233500
19  2020-06-08 1.8424000
20  2020-06-09 0.9611500
21  2020-06-10 0.3125500
22  2020-06-11 0.2679000
23  2020-06-12 0.0000000
24  2020-06-13 0.0000000
25  2020-06-14 0.0000000
26  2020-06-15 0.0000000
27  2020-06-16 0.0000000
28  2020-06-17 0.0000000
29  2020-06-18 0.0000000
30  2020-06-19 0.0041125
31  2020-06-20 0.0041125
32  2020-06-21 0.0035250
33  2020-06-22 0.0000000
34  2020-06-23 0.0000000
35  2020-06-24 0.0000000
71  2020-06-25 0.0000000
81  2020-06-26 0.0230300
91  2020-06-2

Warning message in readLines(paste(dir, "bat.conf", sep = "")):
“incomplete final line found on 'Modelos_Chuva_Vazao_20200624/SMAP/Grande/ARQ_ENTRADA/bat.conf'”

    Area str k2t crec ai capc kkt k2t2  H1   H k3t k1t ecof pcof ecof2 sup_ebin
40 20046 252 3.5  100  1   30 180   10 200 200  10  10 0.81 0.94     0      1.2
   inf_ebin sup_chuva inf_chuva kt_1 kt_2 kt_3 kt_4 kt_5   subbacia  bacia kt_6
40      0.8         2       0.5    0  0.3 0.35 0.35    0 MARIMBONDO Grande   NA
   kt_7 kt_8 kt_9 kt_10 kt_11 kt_12 kt_13 kt_14 kt_15 kt_16 kt_17 kt_18
40   NA   NA   NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
Initializing the virtual microbats...
Finding the best bat
Moving the bats via random walk


date,MARIMBONDO
2020-05-24,155.5554
2020-05-25,155.0935
2020-05-26,154.4735
2020-05-27,153.8602
2020-05-28,153.2528
2020-05-29,152.6506
2020-05-30,152.0531
2020-05-31,151.4598
2020-06-01,150.8704
2020-06-02,150.2845


In [7]:
datename

[1] "20200624"

In [8]:
lag

[1] 1

In [9]:
namesObs[x]

ERROR: Error in eval(expr, envir, enclos): object 'x' not found


In [12]:
namesObs[x]

ERROR: Error in eval(expr, envir, enclos): object 'x' not found
